# Setup for Amazon Bedrock using Local VS Code
- Test Environment: Apple M1 Pro, Ventura 13.5.2


# 1. Pre-requisite (선수 항목)

## 0. Materials
- Bedrock user guide
    - https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-service.html


## 1. role setting (adding trust relationship)

### 1.1. role check

In [1]:
# from sagemaker import get_execution_role

In [2]:
# strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
strSageMakerRoleName = "arn:aws:iam::057716757052:role/bedrock_vscode_gsmoon"
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: arn:aws:iam::057716757052:role/bedrock_vscode_gsmoon


### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 2개의 권한 추가

![nn](../imgs/policy.jpg)

### 1.3. policy
- 아래와 같이 인라인 정책 생성 (Add permissions - Create inline policy)
- policy name: bedrock
![nn](../imgs/inline-policy.jpg)

In [3]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "bedrock:*"
            ],
            "Resource": "*",
            "Effect": "Allow",
            "Sid": "BedrockFullAccess"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Action': ['bedrock:*'],
   'Resource': '*',
   'Effect': 'Allow',
   'Sid': 'BedrockFullAccess'}]}

### 1.4. Trust relationship
![nn](../imgs/trust-relationship.jpg)

In [4]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'Service': 'sagemaker.amazonaws.com'},
   'Action': 'sts:AssumeRole'},
  {'Effect': 'Allow',
   'Principal': {'Service': 'bedrock.amazonaws.com'},
   'Action': 'sts:AssumeRole'}]}

## 2. Model access

### 2.1. Amazon Bedrock Console
![nn](../imgs/model-access/1.png)

### 2.2. "Get Started"
![nn](../imgs/model-access/2.png)

### 2.3. "Model access" - "Edit"
사용 모델 활성화 창
![nn](../imgs/model-access/3.png)

### 2.3. "Save Changes"
사용할 모델을 활성화 후 저장
![nn](../imgs/model-access/4.png)

## 3. Install python SDK for bedrock

In [5]:
install_needed = True

In [2]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awscli
    !{sys.executable} -m pip install -U botocore
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U langchain #==0.0.312
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    !{sys.executable} -m pip install -U librosa
    !{sys.executable} -m pip install -U opensearch-py
    !{sys.executable} -m pip install -U sqlalchemy==2.0.1
    !{sys.executable} -m pip install -U pypdf
    !{sys.executable} -m pip install -U spacy
    !{sys.executable} -m spacy download ko_core_news_md
    !{sys.executable} -m pip install -U ipython

    IPython.Application.instance().kernel.do_shutdown(True)

NameError: name 'install_needed' is not defined

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [6]:
import boto3
import awscli
import botocore
import langchain
from pprint import pprint
from termcolor import colored
from utils.bedrock import bedrock_info

In [7]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")
print(f"botocore version check: {botocore.__version__}")
print(f"awscli version check: {awscli.__version__}")

langchain version check: 0.0.323
boto3 version check: 1.28.71
botocore version check: 1.31.71
awscli version check: 1.29.71


**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [11]:
bedrock_region = "us-east-1"

In [12]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=bedrock_region,
    endpoint_url=f"https://bedrock-runtime.{bedrock_region}.amazonaws.com"
)

In [13]:
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())


== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


## 5. generation

In [14]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### Claude v2

In [18]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=bedrock_client,
    model_kwargs={
        "max_tokens_to_sample":512,
        "stop_sequences":["\n\nHuman:"],
        "temperature":0,
        "top_p":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [23]:
prompt = "\n\nHuman: Please let us know SageMaker's advantages in 100 words \n\nAssistant:"

In [24]:
response = llm(prompt)

 Here is a 100 word summary of some of SageMaker's key advantages:

SageMaker makes it easy to build, train, and deploy machine learning models at scale. It provides pre-built algorithms and frameworks as well as fully managed infrastructure to quickly get models into production. Automated hyperparameter tuning saves time by finding the best model configurations. SageMaker Studio provides a visual IDE for the full ML workflow. And SageMaker Neo compiles models for faster inference. The service is fully managed so you don't have to worry about infrastructure management. These capabilities help data scientists be more productive and get models to market faster.

* Long text to examine 100k-input